# Resume Featching Data from the Database

In [ ]:
# Import python packages
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd

session = st.connection('snowflake').session()

# Write directly to the app
st.title("Resume Screening :book:")
st.write("In this web application, employer can filter out the candidate's resume and rank the resumes by comparing with the job description")

# # Access the Snowflake table using Snowpark's DataFrame API
# def fetch_data_from_snowflake():
#     # Reference the Snowflake table directly
#     snowflake_table = session.table("RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_PREDICTION_CLEAN")
#     # Convert to pandas DataFrame for use with Streamlit
#     df = snowflake_table.to_pandas()  
#     return df

# # Display the data using Streamlit
# st.write("## Resume Prediction")
# st.write("### Fetch data with direct table reference method")

# data = fetch_data_from_snowflake()  # Fetch data
# st.dataframe(data)  # Display it in a table format

# SQL query to fetch specific columns
def fetch_data():
    query = "SELECT full_name, email, job_title FROM RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_PREDICTION_CLEAN"
    # Execute the query and convert the result to a pandas DataFrame
    df = session.sql(query).to_pandas()
    return df
    
st.write("### Fetch data with SQL Query Method")
sql_fetch = fetch_data()  # Fetch data
st.dataframe(sql_fetch)  # Display it in a table format

#view the columns anem
st.write(sql_fetch.columns)

# Streamlit dropdown to filter data by Job Title
job_title_filter = st.selectbox("Select Job Title", options=["All"] + list(sql_fetch['JOB_TITLE'].dropna().unique()))

# Filter the data based on the selected job title
if job_title_filter != "All":
    filtered_df = sql_fetch[sql_fetch['JOB_TITLE'] == job_title_filter]
else:
    filtered_df = sql_fetch

# Display filtered data
st.write(f"### Filtered Resumes based on Job Title: {job_title_filter}")
st.dataframe(filtered_df)

# Resume Profile

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd

# Connect to Snowflake
session = st.connection('snowflake').session()

# SQL query to fetch candidate data from Snowflake
def fetch_data():
    query = """
    SELECT FULL_NAME, PHONE_NUMBER, EMAIL, LOCATION, JOB_TITLE, JOB_LEVEL, WORK_EXPERIENCE_ROLE,
           RECENT_WORK_EXPERIENCE, TECHNICAL_SKILLS, YEARS_OF_EXPERIENCE, DEGREE, FIELD_OF_STUDY,
           UNIVERSITY_NAME, CERTIFICATION, LANGUAGES_SPOKEN
    FROM CANDIDATE_PROFILES
    """
    df = session.sql(query).to_pandas()
    return df

# Fetch data from Snowflake
df = fetch_data()

st.title("Resume Profile")

# Streamlit Multiselect for filtering by Job Title
job_title_filter = st.multiselect("Select Job Title", options=["All"] + sorted(list(df['JOB_TITLE'].dropna().unique())))

# Streamlit Multiselect for filtering by Job Level
job_level_filter = st.multiselect("Select Job Level", options=["All"] + sorted(list(df['JOB_LEVEL'].dropna().unique())))

# Streamlit Multiselect for filtering by Technical Skills
skills_available = sorted(list(set([skill for sublist in df['TECHNICAL_SKILLS'].str.split(', ') for skill in sublist])))  # Extract unique skills
technical_skill_filter = st.multiselect("Select Technical Skill(s)", options=skills_available)

# Streamlit Slider for filtering by Years of Experience
year_of_experience_filter = st.slider("Select Years of Experience", min_value=int(df['YEARS_OF_EXPERIENCE'].min()), 
                                      max_value=int(df['YEARS_OF_EXPERIENCE'].max()), step=1)

# Filter the data based on the selected job titles
filtered_df = df
if "All" not in job_title_filter and job_title_filter:
    filtered_df = filtered_df[filtered_df['JOB_TITLE'].isin(job_title_filter)]

# Filter the data based on the selected job levels
if "All" not in job_level_filter and job_level_filter:
    filtered_df = filtered_df[filtered_df['JOB_LEVEL'].isin(job_level_filter)]
    
# Filter by technical skills (match at least one of the selected skills)
if technical_skill_filter:
    filtered_df = filtered_df[filtered_df['TECHNICAL_SKILLS'].apply(lambda skills: any(skill in skills.split(', ') for skill in technical_skill_filter))]

# Filter by years of experience
filtered_df = filtered_df[filtered_df['YEARS_OF_EXPERIENCE'] >= year_of_experience_filter]

# Display filtered data as a table
st.write(f"### Filtered Resumes")
st.dataframe(filtered_df)

# Profile Cards Styling
for i, row in filtered_df.iterrows():
    with st.expander(f"{row['FULL_NAME']} - {row['JOB_TITLE']}"):
        st.subheader(f"📧 {row['EMAIL']}")
        st.write(f"**Phone:** {row['PHONE_NUMBER']}")
        st.write(f"**Location:** {row['LOCATION']}")
        st.write(f"**Job Level:** {row['JOB_LEVEL']}")
        st.write(f"**Work Experience Role:** {row['WORK_EXPERIENCE_ROLE']}")
        st.write(f"**Recent Work Experience:** {row['RECENT_WORK_EXPERIENCE']}")
        
        # Highlight technical skills with colored boxes in a single row
        st.write(f"**Technical Skills:**")
        skills = row['TECHNICAL_SKILLS'].split(', ')
        skills_html = "".join([f"<span style='background-color: #034eda; color: white; padding: 5px 10px; border-radius: 4px; margin-right: 8px;'>{skill}</span>" for skill in skills])
        st.markdown(f"<div>{skills_html}</div>", unsafe_allow_html=True)

        st.write("") # indent space
        st.write(f"**Years of Experience:** {row['YEARS_OF_EXPERIENCE']}")
        st.write(f"**Degree:** {row['DEGREE']}")
        st.write(f"**Field of Study:** {row['FIELD_OF_STUDY']}")

# Combine resume profile with editing feature 

In [ ]:
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session
import plotly.express as px

# Set wide layout
#st.set_page_config(layout="wide")

# Widen the Streamlit canvas
st.markdown("""
    <style>
        .main .block-container {
            max-width: 95%;
            padding-top: 1rem;
            padding-right: 2rem;
            padding-left: 2rem;
        }
    </style>
""", unsafe_allow_html=True)

# Connect to Snowflake
session = get_active_session()

# Fetch data from Snowflake
@st.cache_data
def fetch_data():
    query = """
    SELECT FULL_NAME, PHONE_NUMBER, EMAIL, LOCATION, JOB_TITLE, JOB_LEVEL, WORK_EXPERIENCE_ROLE,
           RECENT_WORK_EXPERIENCE, TECHNICAL_SKILLS, YEARS_OF_EXPERIENCE, DEGREE, FIELD_OF_STUDY,
           UNIVERSITY_NAME, CERTIFICATION, LANGUAGES_SPOKEN
    FROM CANDIDATE_PROFILES
    """
    return session.sql(query).to_pandas()

df = fetch_data()

# === Sidebar Filters ===
st.sidebar.header("Filters")
job_titles = df['JOB_TITLE'].dropna().unique()
job_levels = df['JOB_LEVEL'].dropna().unique()
all_skills = sorted({skill.strip() for skills in df['TECHNICAL_SKILLS'].fillna('').tolist() for skill in skills.split(',') if skill.strip()})
min_exp = 1
max_exp = int(df['YEARS_OF_EXPERIENCE'].max())

selected_job_titles = st.sidebar.multiselect("Job Title", job_titles)
selected_job_levels = st.sidebar.multiselect("Job Level", job_levels)
selected_skills = st.sidebar.multiselect("Technical Skills", all_skills)
selected_exp = st.sidebar.slider("Minimum Years of Experience", min_value=min_exp, max_value=max_exp, value=min_exp)

# === Apply Filters ===
filtered_df = df.copy()
if selected_job_titles:
    filtered_df = filtered_df[filtered_df['JOB_TITLE'].isin(selected_job_titles)]

if selected_job_levels:
    filtered_df = filtered_df[filtered_df['JOB_LEVEL'].isin(selected_job_levels)]

filtered_df = filtered_df[filtered_df['YEARS_OF_EXPERIENCE'] >= selected_exp]

# #This checks if at least one of the selected_skills is contained somewhere in the 'TECHNICAL_SKILLS'
# if selected_skills:
#     filtered_df = filtered_df[
#         filtered_df['TECHNICAL_SKILLS'].apply(lambda skills: any(skill in skills for skill in selected_skills))
#     ]

#This checks if all the selected skills match with all the skills in the technical skills column
filtered_df = filtered_df[
    filtered_df['TECHNICAL_SKILLS'].apply(
        lambda s: set(selected_skills).issubset(set(map(str.strip, s.split(','))))
    )
]

# === Display Cards ===
st.write("# Candidate Resume Profile")


# Donut Chart: Job Title Distribution
job_title_counts = filtered_df['JOB_TITLE'].value_counts().reset_index()
job_title_counts.columns = ['Job Title', 'Count']

if not job_title_counts.empty:
    fig = px.pie(job_title_counts, names='Job Title', values='Count',
                 title='Distribution of Job Titles',
                 hole=0.4,  # Creates a donut chart by setting the hole size
                 color_discrete_sequence=px.colors.qualitative.Pastel)

    # Update the chart to display the count labels
    fig.update_traces(textinfo='label+value', textposition='outside', pull=[0.05, 0.05, 0.05, 0.05])
    fig.update_layout(
        margin=dict(t=0, b=0, l=0, r=0),  # Reduce margin to make it more compact
        height=200  # Resize the chart to be more compact
    )
    
    st.plotly_chart(fig, use_container_width=True)
else:
    st.info("No data available to show in the donut chart based on current filters.")
    


for idx, row in filtered_df.iterrows():
    skill_list = [s.strip() for s in row['TECHNICAL_SKILLS'].split(',') if s.strip()]
    
    job_title_colors = {
        "Data Scientist": "#b3e5fc", "Data Analyst": "#ffccbc", "BI Consultant": "#c8e6c9", "Data Scientist ": "#f0f4c3"
    }
    job_level_colors = {
        "Junior": "#ffe082", "Mid-level": "#ffeb3b", "Senior": "#f57c00"
    }
    
    job_title_color = job_title_colors.get(row['JOB_TITLE'], "#e0f7fa")
    job_level_color = job_level_colors.get(row['JOB_LEVEL'], "#fce4ec")

    st.markdown(
        f"""
        <div style='
            border: 1px solid #ccc;
            border-radius: 5px;
            padding: 1px;
            margin-bottom: 20px;
            background-color: #f9f9f9;
        '>
        """,
        unsafe_allow_html=True
    )

    cols = st.columns([3, 2, 3, 1])
    with cols[0]:
        st.markdown(f"**{row['FULL_NAME']}**")
        st.markdown(
            f"""
            <span style='background-color:{job_title_color}; color:#006064; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:13px'>
                {row['JOB_TITLE']}
            </span>
            <span style='background-color:{job_level_color}; color:#880e4f; padding:4px 8px; border-radius:8px; font-size:13px'>
                {row['JOB_LEVEL']}
            </span>
            """,
            unsafe_allow_html=True
        )
    with cols[1]:
        st.markdown(f"**Experience:** {row['YEARS_OF_EXPERIENCE']} years")
    with cols[2]:
        st.markdown("**Skills:**")
        skill_colors = ["#a5d6a7", "#c5cae9", "#ffcc80", "#ffab91", "#80deea", "#f48fb1"]
        skill_badges = " ".join(
            [f"<span style='background-color:{skill_colors[i % len(skill_colors)]}; color:#2e7d32; padding:3px 6px; border-radius:6px; font-size:12px; margin:2px'>{s}</span>" for i, s in enumerate(skill_list)]
        )
        st.markdown(f"<div style='display: flex; flex-wrap: wrap'>{skill_badges}</div>", unsafe_allow_html=True)
    with cols[3]:
        edit_key = f"edit_{idx}"
        if st.button("Edit", key=edit_key):
            st.session_state[f"edit_card_{idx}"] = not st.session_state.get(f"edit_card_{idx}", False)
    # Update record function
    # Might need to use candiate's ID instead of the name for unique identifier 
    def update_candidate_profile(name, updated_values):
        set_clause = ", ".join([
            f"{col} = '{val}'" if isinstance(val, str) else f"{col} = {val}"
            for col, val in updated_values.items()
        ])
        query = f"""
        UPDATE CANDIDATE_PROFILES
        SET {set_clause}
        WHERE FULL_NAME = '{name}'
        """
        session.sql(query).collect()

    if st.session_state.get(f"edit_card_{idx}", False):
        with st.expander(f"Edit: {row['FULL_NAME']}", expanded=True):
            new_job_title = st.text_input("Job Title", value=row['JOB_TITLE'], key=f"title_{idx}")
            new_job_level = st.text_input("Job Level", value=row['JOB_LEVEL'], key=f"level_{idx}")
            new_experience = st.number_input("Years of Experience", min_value=0, value=int(row['YEARS_OF_EXPERIENCE']), key=f"exp_{idx}")
            new_skills = st.text_input("Technical Skills (comma-separated)", value=row['TECHNICAL_SKILLS'], key=f"skills_{idx}")

            if st.button("Submit Changes", key=f"submit_{idx}"):
                updated_values = {
                    'JOB_TITLE': new_job_title,
                    'JOB_LEVEL': new_job_level,
                    'YEARS_OF_EXPERIENCE': new_experience,
                    'TECHNICAL_SKILLS': new_skills
                }
                update_candidate_profile(row['FULL_NAME'], updated_values)
                st.success(f"{row['FULL_NAME']}'s profile updated!")
                st.cache_data.clear()
                st.rerun()

    # Close card container
    st.markdown("</div>", unsafe_allow_html=True)

# Resume Profile with Validation Filter, Updating Validation Status, Application Status, Remark

In [ ]:
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session
import plotly.express as px

# Set wide layout
st.set_page_config(layout="wide")

# Widen the Streamlit canvas
st.markdown("""
    <style>
        .main .block-container {
            max-width: 95%;
            padding-top: 1rem;
            padding-right: 2rem;
            padding-left: 2rem;
        }
    </style>
""", unsafe_allow_html=True)

# Connect to Snowflake
session = get_active_session()

# Fetch data from Snowflake
@st.cache_data
def fetch_data():
    query = """
    SELECT FULL_NAME, PHONE_NUMBER, EMAIL, LOCATION, JOB_TITLE, JOB_LEVEL, WORK_EXPERIENCE_ROLE,
           RECENT_WORK_EXPERIENCE, TECHNICAL_SKILLS, YEARS_OF_EXPERIENCE, DEGREE, FIELD_OF_STUDY,
           UNIVERSITY_NAME, CERTIFICATION, LANGUAGES_SPOKEN, CONFIDENCE_SCORE, VALIDATION_STATUS, 
           APPLICATION_STATUS, REMARK
    FROM CANDIDATE_PROFILES_EDIT
    """
    return session.sql(query).to_pandas()

df = fetch_data()

# Count how many candidates need validation
num_validation = (df["VALIDATION_STATUS"] == "Need Validation").sum()

# Show it in Streamlit sidebar
st.sidebar.markdown(f"""
<span style='font-size:18px;'>🚨 Validation Needed: <strong style='font-size:20px; font-weight:bold; color:#d62828;'>{num_validation}</strong></span>
""", unsafe_allow_html=True)

# === Sidebar Filters ===
st.sidebar.header("Filters")
job_titles = df['JOB_TITLE'].dropna().unique()
job_levels = df['JOB_LEVEL'].dropna().unique()
all_skills = sorted({skill.strip() for skills in df['TECHNICAL_SKILLS'].fillna('').tolist() for skill in skills.split(',') if skill.strip()})
min_exp = 1
max_exp = int(df['YEARS_OF_EXPERIENCE'].max())

# Unique validation statuses from the dataframe
validation_options = ["All", "Validated", "Need Validation", "Pass Score"]

# Unique validation statuses from the dataframe
application_options = ["All", "Applied ", "Under Review", "Shortlisted", "Interview", "Rejected", "Hired"]

selected_job_titles = st.sidebar.multiselect("Job Title", job_titles)
selected_job_levels = st.sidebar.multiselect("Job Level", job_levels)
selected_skills = st.sidebar.multiselect("Technical Skills", all_skills)
selected_exp = st.sidebar.slider("Minimum Years of Experience", min_value=min_exp, max_value=max_exp, value=min_exp)

# Sidebar filter for validation
selected_validation = st.sidebar.selectbox("Validation Status", validation_options)

# Sidebar filter for validation
selected_application = st.sidebar.selectbox("Application Status", application_options)

# === Apply Filters ===
filtered_df = df.copy()
if selected_job_titles:
    filtered_df = filtered_df[filtered_df['JOB_TITLE'].isin(selected_job_titles)]
if selected_job_levels:
    filtered_df = filtered_df[filtered_df['JOB_LEVEL'].isin(selected_job_levels)]
filtered_df = filtered_df[filtered_df['YEARS_OF_EXPERIENCE'] >= selected_exp]
filtered_df = filtered_df[
    filtered_df['TECHNICAL_SKILLS'].apply(
        lambda s: set(selected_skills).issubset(set(map(str.strip, s.split(','))))
    )
]
#Filter validation status
if selected_validation != "All":
    filtered_df = filtered_df[filtered_df['VALIDATION_STATUS'] == selected_validation]

#Filter application status
if selected_application != "All":
    filtered_df = filtered_df[filtered_df['APPLICATION_STATUS'] == selected_application]


# === Display Cards ===
st.write("# Candidate Resume Profile")

# Donut Chart: Job Title Distribution
job_title_counts = filtered_df['JOB_TITLE'].value_counts().reset_index()
job_title_counts.columns = ['Job Title', 'Count']

if not job_title_counts.empty:
    fig = px.pie(job_title_counts, names='Job Title', values='Count',
                 title='Distribution of Job Titles',
                 hole=0.4,
                 color_discrete_sequence=px.colors.qualitative.Pastel)

    fig.update_traces(textinfo='label+value', textposition='outside', pull=[0.05] * len(job_title_counts))
    fig.update_layout(margin=dict(t=0, b=0, l=0, r=0), height=200)
    st.plotly_chart(fig, use_container_width=True)
else:
    st.info("No data available to show in the donut chart based on current filters.")

for idx, row in filtered_df.iterrows():
    skill_list = [s.strip() for s in row['TECHNICAL_SKILLS'].split(',') if s.strip()]
    
    job_title_colors = {
        "Data Scientist": "#b3e5fc", "Data Analyst": "#ffccbc", "BI Consultant": "#c8e6c9", "Data Scientist ": "#f0f4c3"
    }
    job_level_colors = {
        "Junior": "#ffe082", "Mid-level": "#ffeb3b", "Senior": "#f57c00"
    }

    application_status_colors = {
    "Applied": "#e3f2fd",        # Light Blue
    "Under Review": "#fff9c4",   # Light Yellow
    "Shortlisted": "#c8e6c9",    # Light Green
    "Interview": "#b3e5fc",      # Sky Blue
    "Rejected": "#ffcdd2",       # Light Red
    "Hired": "#dcedc8"           # Light Lime Green
    }
    
    job_title_color = job_title_colors.get(row['JOB_TITLE'], "#e0f7fa")
    job_level_color = job_level_colors.get(row['JOB_LEVEL'], "#fce4ec")
    application_status_colors = application_status_colors.get(row['APPLICATION_STATUS'], "#e0e0e0")
    
    with st.container():
        st.markdown(
            f"""
            <div style='
                border: 1px solid #ddd;
                border-radius: 8px;
                padding: 5px 15px;
                margin-bottom: 10px;
                background-color: #fff;
            '>
            """,
            unsafe_allow_html=True
        )
        
        inner_cols = st.columns([3, 3, 2, .5])
        with inner_cols[0]:
            st.markdown(f"### {row['FULL_NAME']}")
            st.markdown(f"**Experience:** {row['YEARS_OF_EXPERIENCE']} years")
            
            # Determine the badge to show based on validation status
            if row['VALIDATION_STATUS'] == "Need Validation":
                validation_badge = "<span style='background-color:#f8d7da; color:#c62828; padding:4px 8px; border-radius:8px; font-size:12px; margin-left:6px'>❌ Need Validation</span>"
            elif row['VALIDATION_STATUS'] == "Validated":
                validation_badge = "<span style='background-color:#d0f0c0; color:#2e7d32; padding:4px 8px; border-radius:8px; font-size:12px; margin-left:6px'>✅ Validated</span>"
            else:
                validation_badge = "<span style='background-color:#e3f2fd; color:#0277bd; padding:4px 8px; border-radius:8px; font-size:12px; margin-left:6px'>ℹ️ Pass Score</span>"
            
            # Then insert it into your HTML
            st.markdown(
                f"""
                <div style="margin-top: 1px;">
                    <span style='background-color:{job_title_color}; color:#006064; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>
                        {row['JOB_TITLE']}
                    </span>
                    <span style='background-color:{job_level_color}; color:#880e4f; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>
                        {row['JOB_LEVEL']}
                    </span>
                    {validation_badge}
                </div>
                """,
                unsafe_allow_html=True
            )

        
        with inner_cols[1]:
            st.markdown(f"**Skills:**")
            skill_colors = ["#a5d6a7", "#c5cae9", "#ffcc80", "#ffab91", "#80deea", "#f48fb1"]
            skill_badges = " ".join([
                f"<span style='background-color:{skill_colors[i % len(skill_colors)]}; color:#2e7d32; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>{s}</span>"
                for i, s in enumerate(skill_list)
            ])
            st.markdown(f"<div style='margin-top:10px; display: flex; flex-wrap: wrap'>{skill_badges}</div>", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)

        
        with inner_cols[2]:
            st.markdown(f"**Application Status:**")
            st.markdown(
                f"""
                <div style="margin-top: 10px;">
                    <span style='background-color:{application_status_colors}; color:#006064; padding:4px 8px; border-radius:8px; margin-right:6px; font-size:12px'>
                        {row['APPLICATION_STATUS']}
                    </span>
                </div>
                """,
                unsafe_allow_html=True
            )
        
        with inner_cols[3]:
            edit_key = f"edit_{idx}"
            if st.button("Edit", key=edit_key):
                st.session_state[f"edit_card_{idx}"] = not st.session_state.get(f"edit_card_{idx}", False)

        st.markdown("</div>", unsafe_allow_html=True)

    def update_candidate_profile(name, updated_values):
        set_clause = ", ".join([
            f"{col} = '{val}'" if isinstance(val, str) else f"{col} = {val}"
            for col, val in updated_values.items()
        ])
        query = f"""
        UPDATE CANDIDATE_PROFILES_EDIT
        SET {set_clause}
        WHERE FULL_NAME = '{name}'
        """
        session.sql(query).collect()

    if st.session_state.get(f"edit_card_{idx}", False):
        with st.expander(f"Edit: {row['FULL_NAME']}", expanded=True):
            new_job_title = st.text_input("Job Title", value=row['JOB_TITLE'], key=f"title_{idx}")
            new_job_level = st.text_input("Job Level", value=row['JOB_LEVEL'], key=f"level_{idx}")
            new_experience = st.number_input("Years of Experience", min_value=0, value=int(row['YEARS_OF_EXPERIENCE']), key=f"exp_{idx}")
            new_skills = st.text_input("Technical Skills (comma-separated)", value=row['TECHNICAL_SKILLS'], key=f"skills_{idx}")

            # Dropdown for Validation Status
            validation_status_options = ["Need Validation", "Validated", "Pass Score"]
            new_validation_status = st.selectbox(
                "Validation Status", 
                options=validation_status_options,
                index=validation_status_options.index(row['VALIDATION_STATUS']) if row['VALIDATION_STATUS'] in validation_status_options else 0,
                key=f"vali_status_{idx}"
            )
            
            # Dropdown for Application Status
            application_status_options = ["Applied ", "Under Review", "Shortlisted", "Interview", "Rejected", "Hired"]
            new_application_status = st.selectbox(
                "Application Status",
                options=application_status_options,
                index=application_status_options.index(row['APPLICATION_STATUS']) if row['APPLICATION_STATUS'] in application_status_options else 0,
                key=f"appli_status_{idx}"
            )
    
            new_remark = st.text_input("Validation Remark", value=row['REMARK'], key=f"remark_{idx}")
    
            
            if st.button("Submit Changes", key=f"submit_{idx}"):
                updated_values = {
                    'JOB_TITLE': new_job_title,
                    'JOB_LEVEL': new_job_level,
                    'YEARS_OF_EXPERIENCE': new_experience,
                    'TECHNICAL_SKILLS': new_skills,
                    'VALIDATION_STATUS': new_validation_status,
                    'APPLICATION_STATUS': new_application_status,
                    'REMARK': new_remark
                }
                update_candidate_profile(row['FULL_NAME'], updated_values)
                st.success(f"{row['FULL_NAME']}'s profile updated!")
                st.cache_data.clear()
                st.rerun()

    st.markdown("</div>", unsafe_allow_html=True)